<a href="https://colab.research.google.com/github/Risad-Raihan/ops_segmentation/blob/main/OPS_regular_seg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)


Mounted at /content/drive


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/content/drive/My Drive/ops_segmentation-main/sale_data.csv')

In [4]:

df.head(5)

,Date,customer_code,outlet,quantity,sales_price
0,2023-01-02,AP01500566,Saturn Textiles Ltd.,647.0,169210.00
1,2023-01-04,AP01005995,Southern Garments Ltd.,148.0,5989.00
2,2023-01-31,AP02733858,Abanti Colour Tex Limited,129.0,3054.80
3,2023-02-02,AP01005995,Southern Garments Ltd.,8.0,2940.00
4,2023-02-02,AP02733858,Abanti Colour Tex Limited,132.0,2899.02


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 458004 entries, 0 to 458003
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           458004 non-null  object 
 1   customer_code  458004 non-null  object 
 2   outlet         457972 non-null  object 
 3   quantity       458004 non-null  float64
 4   sales_price    458004 non-null  float64
dtypes: float64(2), object(3)
memory usage: 17.5+ MB


In [13]:
import pandas as pd

# Ensure the 'Date' column across all DataFrames is in datetime format
df['Date'] = pd.to_datetime(df['Date'])
df_angel['Date'] = pd.to_datetime(df_angel['Date'])
df_fresh['Date'] = pd.to_datetime(df_fresh['Date'])

# Merge df_angel and df_fresh to form a DataFrame of excluded customers
excluded_customers = pd.concat([df_angel, df_fresh]).drop_duplicates()

# Include the 'outlet' in the merge to retain this information
df_return = df.merge(excluded_customers, on=['Date', 'customer_code', 'outlet'], how='left', indicator=True)

# Filter to keep only those rows where '_merge' is 'left_only' indicating the customer is not in excluded_customers
df_return = df_return[df_return['_merge'] == 'left_only']

# Select only the Date, customer_code, and outlet columns for the final DataFrame
df_return = df_return[['Date', 'customer_code', 'outlet']]

# Display the DataFrame of returning customers
print(df_return)


<ipython-input-13-601d862c2641>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_angel['Date'] = pd.to_datetime(df_angel['Date'])


             Date customer_code                     outlet
8      2023-02-20    AP02733856  Abanti Colour Tex Limited
9      2023-02-22    AP02733856  Abanti Colour Tex Limited
63     2023-02-12    AP02733856  Abanti Colour Tex Limited
66     2023-02-19    AP02733856  Abanti Colour Tex Limited
117    2023-02-14    AP02733856  Abanti Colour Tex Limited
...           ...           ...                        ...
457991 2024-09-29    SSL1188277         Silken Sewing Ltd.
457992 2024-09-29    SSL1228226         Silken Sewing Ltd.
457993 2024-09-29    SSL1228341         Silken Sewing Ltd.
457998 2024-09-29    SSL1431605         Silken Sewing Ltd.
458000 2024-09-29    SSL1431674         Silken Sewing Ltd.

[103541 rows x 3 columns]


In [14]:
df_return.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103541 entries, 8 to 458000
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Date           103541 non-null  datetime64[ns]
 1   customer_code  103541 non-null  object        
 2   outlet         103520 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 3.2+ MB


In [7]:
import pandas as pd

# Assuming df is loaded and the 'Date' column is appropriately formatted
df['Date'] = pd.to_datetime(df['Date'])

# Define the sales threshold
sales_threshold = 1399

# Pre-compute the total sales per customer per outlet per month
monthly_sales = df.groupby(['customer_code', 'outlet', pd.Grouper(key='Date', freq='M')]).agg(total_sales=('sales_price', 'sum')).reset_index()

# Identify months with valid sales over the threshold
valid_sales = monthly_sales[monthly_sales['total_sales'] > sales_threshold]

# Count the number of valid months per customer per outlet
valid_months_count = valid_sales.groupby(['customer_code', 'outlet']).size().reset_index(name='valid_months')

# Filter customers who meet the criteria across all dates
qualifying_customers = valid_months_count[valid_months_count['valid_months'] >= 2]

# Prepare the DataFrame for merging
# Convert 'Month' from the 'valid_sales' for merging
valid_sales['Month'] = valid_sales['Date'].dt.to_period('M')

# Merge this data with df to find actual days where transactions exist for these customers
df['Month'] = df['Date'].dt.to_period('M')
qualified_daily = df.merge(qualifying_customers, on=['customer_code', 'outlet'], how='inner')
qualified_daily = qualified_daily[qualified_daily['Month'] == qualified_daily['Date'].dt.to_period('M')]

# Drop duplicates to ensure unique entries
qualified_daily.drop_duplicates(subset=['Date', 'customer_code', 'outlet'], inplace=True)

# Select relevant columns
df_angel = qualified_daily[['Date', 'customer_code', 'outlet']]

print(df_angel)


<ipython-input-7-a66c743f83b8>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_sales['Month'] = valid_sales['Date'].dt.to_period('M')


             Date customer_code                     outlet
0      2023-01-02    AP01500566       Saturn Textiles Ltd.
1      2023-02-05    AP01500566       Saturn Textiles Ltd.
2      2023-04-03    AP01500566       Saturn Textiles Ltd.
3      2023-01-08    AP01500566       Saturn Textiles Ltd.
4      2023-02-13    AP01500566       Saturn Textiles Ltd.
...           ...           ...                        ...
283430 2024-09-17  AP0200525931  Majumder Garments Limited
283431 2024-08-14  AP0320001910            KDS IDR Limited
283432 2024-09-17  AP0320001910            KDS IDR Limited
283433 2024-08-25  AP0200525179  Majumder Garments Limited
283434 2024-09-12  AP0200525179  Majumder Garments Limited

[283435 rows x 3 columns]


In [8]:
import pandas as pd

# Assuming 'df' is already loaded and the 'Date' column is formatted as datetime
# Convert Date column if it's not already in datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

# Create a new DataFrame to store each customer's first purchase date
first_purchases = df.groupby('customer_code')['Date'].min().reset_index()
first_purchases.rename(columns={'Date': 'First_Purchase_Date'}, inplace=True)

# Merge this information back to the original DataFrame to annotate each transaction
df = df.merge(first_purchases, on='customer_code', how='left')

# Filter transactions where the transaction date is the same as the first purchase date
df_fresh = df[df['Date'] == df['First_Purchase_Date']]

# Select only the necessary columns for df_fresh
df_fresh = df_fresh[['Date', 'customer_code', 'outlet']].drop_duplicates()

# Display the resulting DataFrame of fresh customers
print(df_fresh)


             Date customer_code                outlet
0      2023-01-02    AP01500566  Saturn Textiles Ltd.
26     2023-05-18  AP0060000014            SQ Station
27     2023-05-18  AP0060000986            SQ Station
28     2023-05-18  AP0060001450            SQ Station
29     2023-05-18  AP0060002906            SQ Station
...           ...           ...                   ...
457996 2024-09-29    SSL1391460    Silken Sewing Ltd.
457997 2024-09-29    SSL1391493    Silken Sewing Ltd.
458001 2024-09-29    SSL1596193    Silken Sewing Ltd.
458002 2024-09-29    SSL1840720    Silken Sewing Ltd.
458003 2024-09-29    SSL2054572    Silken Sewing Ltd.

[90813 rows x 3 columns]


In [12]:
df_fresh.info()

<class 'pandas.core.frame.DataFrame'>
Index: 90813 entries, 0 to 458003
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Date           90813 non-null  datetime64[ns]
 1   customer_code  90813 non-null  object        
 2   outlet         90802 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 2.8+ MB


In [15]:
import pandas as pd

# First, aggregate the base data from df for customer count, sales, and quantity
base_agg = df.groupby(['Date', 'outlet']).agg(
    customer_count=('customer_code', 'nunique'),
    total_sales=('sales_price', 'sum'),
    quantity=('quantity', 'sum')
).reset_index()

# Aggregate unique customer counts for angel, fresh, and return customers for each day and outlet
angel_agg = df_angel.groupby(['Date', 'outlet']).agg(
    Angel_count=('customer_code', 'nunique')
).reset_index()

fresh_agg = df_fresh.groupby(['Date', 'outlet']).agg(
    fresh_count=('customer_code', 'nunique')
).reset_index()

return_agg = df_return.groupby(['Date', 'outlet']).agg(
    return_count=('customer_code', 'nunique')
).reset_index()

# Merge these aggregations with the base dataframe
df_final = pd.merge(base_agg, angel_agg, on=['Date', 'outlet'], how='left')
df_final = pd.merge(df_final, fresh_agg, on=['Date', 'outlet'], how='left')
df_final = pd.merge(df_final, return_agg, on=['Date', 'outlet'], how='left')

# Fill NaN values with 0 where there are no customers of that type on that day
df_final.fillna({'Angel_count': 0, 'fresh_count': 0, 'return_count': 0}, inplace=True)

# Ensure all counts are integers
df_final['Angel_count'] = df_final['Angel_count'].astype(int)
df_final['fresh_count'] = df_final['fresh_count'].astype(int)
df_final['return_count'] = df_final['return_count'].astype(int)

# Display the resulting DataFrame
print(df_final)


           Date                     outlet  customer_count  total_sales  \
0    2023-01-02       Saturn Textiles Ltd.               1  169210.0000   
1    2023-01-03     Southern Garments Ltd.               1    1494.0000   
2    2023-01-04       Saturn Textiles Ltd.               1   50085.0000   
3    2023-01-04     Southern Garments Ltd.               1    5989.0000   
4    2023-01-05       Saturn Textiles Ltd.               1   47930.0000   
...         ...                        ...             ...          ...   
7886 2024-09-29  Renaissance Apparels Ltd.              44   17793.4760   
7887 2024-09-29       Saturn Textiles Ltd.              41   16830.4000   
7888 2024-09-29         Silken Sewing Ltd.             153  119042.6135   
7889 2024-09-29     Southern Garments Ltd.              40   18639.0000   
7890 2024-09-29  Ultimate Fashions Limited              38   26236.0000   

      quantity  Angel_count  fresh_count  return_count  
0     647.0000            1            1  

In [16]:
import pandas as pd

# Assuming df_final is already created and loaded

# Drop the existing 'return_count' column
df_final.drop(columns='return_count', inplace=True)

# Create a new 'return_count' column
df_final['return_count'] = df_final['customer_count'] - (df_final['Angel_count'] + df_final['fresh_count'])

# Ensure no negative values in 'return_count'
df_final['return_count'] = df_final['return_count'].clip(lower=0)

# Display the updated DataFrame
print(df_final)


           Date                     outlet  customer_count  total_sales  \
0    2023-01-02       Saturn Textiles Ltd.               1  169210.0000   
1    2023-01-03     Southern Garments Ltd.               1    1494.0000   
2    2023-01-04       Saturn Textiles Ltd.               1   50085.0000   
3    2023-01-04     Southern Garments Ltd.               1    5989.0000   
4    2023-01-05       Saturn Textiles Ltd.               1   47930.0000   
...         ...                        ...             ...          ...   
7886 2024-09-29  Renaissance Apparels Ltd.              44   17793.4760   
7887 2024-09-29       Saturn Textiles Ltd.              41   16830.4000   
7888 2024-09-29         Silken Sewing Ltd.             153  119042.6135   
7889 2024-09-29     Southern Garments Ltd.              40   18639.0000   
7890 2024-09-29  Ultimate Fashions Limited              38   26236.0000   

      quantity  Angel_count  fresh_count  return_count  
0     647.0000            1            1  

In [17]:
# Save DataFrame to Excel file in Colab's local environment
df_final.to_excel('/content/seg_new.xlsx', index=False)
